In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv


# 1 head files

In [2]:
import pandas as pd
import jieba
import pickle
import gensim
import numpy as np
from tqdm import tqdm
import re
import string
import nltk
from nltk.corpus import stopwords

import os
import datetime
import numpy as np
import random
from scipy.sparse import hstack


from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection


# 2 Reading CSV files

In [3]:
label_set = pd.read_csv('../input/nlp-getting-started/train.csv',encoding='gbk',keep_default_na=False)
label_set = label_set.reset_index(drop=True)
print(label_set.text)
print("Training data shape:",label_set.shape)
label_set.head()

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object
Training data shape: (7613, 5)


,id,keyword,location,text,target
0,1,,,Our Deeds are the Reason of this #earthquake M...,1
1,4,,,Forest fire near La Ronge Sask. Canada,1
2,5,,,All residents asked to 'shelter in place' are ...,1
3,6,,,"13,000 people receive #wildfires evacuation or...",1
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# The traing data has 7613 observations and 5 features including the TARGET

In [5]:
#Testing data
test_label_set = pd.read_csv('../input/nlp-getting-started/test.csv',encoding='gbk',keep_default_na=False)
tset_label_set = label_set.reset_index(drop=True)
print(test_label_set.text)
print("Training data shape:",test_label_set.shape)
test_label_set.head()

0                      Just happened a terrible car crash
1       Heard about #earthquake is different cities, s...
2       there is a forest fire at spot pond, geese are...
3                Apocalypse lighting. #Spokane #wildfires
4           Typhoon Soudelor kills 28 in China and Taiwan
                              ...                        
3258    EARTHQUAKE SAFETY LOS ANGELES 聣脹脪 SAFETY FASTE...
3259    Storm in RI worse than last hurricane. My city...
3260    Green Line derailment in Chicago http://t.co/U...
3261    MEG issues Hazardous Weather Outlook (HWO) htt...
3262    #CityofCalgary has activated its Municipal Eme...
Name: text, Length: 3263, dtype: object
Training data shape: (3263, 4)


,id,keyword,location,text
0,0,,,Just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s..."
2,3,,,"there is a forest fire at spot pond, geese are..."
3,9,,,Apocalypse lighting. #Spokane #wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan


# 3 Data preprocessing

## 1 data cleaning

In [6]:
#removing some symbols
#training data
def clean_text(text):
    sentences = text.lower()
    sentences = re.sub (r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', sentences, flags=re.MULTILINE)
    sentences = re.sub('[!#?,.:";//|''@-]','',sentences)
    return sentences
    
label_set['text'] = label_set['text'].apply(lambda x: clean_text(x))
test_label_set['text'] = test_label_set['text'].apply(lambda x:clean_text(x))

all_text = pd.concat([label_set,test_label_set])
all_text.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


,id,keyword,location,target,text
0,1,,,1.0,our deeds are the reason of this earthquake ma...
1,4,,,1.0,forest fire near la ronge sask canada
2,5,,,1.0,all residents asked to 'shelter in place' are ...
3,6,,,1.0,13000 people receive wildfires evacuation orde...
4,7,,,1.0,just got sent this photo from ruby alaska as s...


# 4 Transforming features to a vector

In [7]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
train_features = tfidf.fit_transform(label_set['text'])
test_features = tfidf.transform(test_label_set["text"])
print(train_features.shape)
print(test_features.shape)

(7613, 16953)
(3263, 16953)


# 5 Building a Text Classification model

## Naives Bayes Classifier

In [8]:
nb = MultinomialNB()
scores = model_selection.cross_val_score(nb,train_features,label_set["target"],cv=5,scoring="f1")
scores

array([0.58090452, 0.57327189, 0.62686567, 0.60991581, 0.75163399])

## predict

In [9]:
nb.fit(train_features,label_set['target'])
submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
submission['target'] = nb.predict(test_features)

submission.to_csv("submission.csv",index = False)